<a href="https://colab.research.google.com/github/sngo/llms-practice/blob/main/GPT_Anthropic_Gemini_Concept.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anthropic==0.49.0

In [ ]:
#!pip install --upgrade anthropic

In [ ]:
!pip show anthropic

In [ ]:
!pip install --upgrade httpx

In [ ]:
!pip show httpx

In [6]:
import os
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
from google.colab import userdata
import time

In [7]:
import google.generativeai

In [8]:
openai_api_key = userdata.get('OPENAI_API_KEY')
anthropic_api_key = userdata.get('ANTHROPIC_API_KEY')
google_api_key = userdata.get('GOOGLE_API_KEY')

os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['ANTHROPIC_API_KEY'] = anthropic_api_key
os.environ['GOOGLE_API_KEY'] = google_api_key

In [9]:
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyBy


In [10]:
openai = OpenAI()

claude = anthropic.Anthropic(api_key=anthropic_api_key )

In [11]:
google.generativeai.configure()

In [12]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [13]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [14]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

Why did the data scientist go to therapy?
Because their data was so messy, they needed some normalization!


In [15]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician?

Because she found him too mean!


In [16]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the logistic regression model?

It had too many issues with commitment!


In [17]:
# Claude 3.7 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-7-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

Why don't data scientists like to go to the beach?

Because they're afraid of overfitting their sunscreen application to the training data and getting burned on the test data!


In [18]:
# Claude 3.7 Sonnet again
# Now let's add in streaming back results
# If the streaming looks strange, then please see the note below this cell!

result = claude.messages.stream(
    model="claude-3-7-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)
            time.sleep(0.02)

Why don't data scientists like to play hide and seek?

Because they always find you... with 99.7% confidence interval!

In [19]:
# The API for Gemini has a slightly different structure.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash',
    system_instruction=system_message
)

try:
  response = gemini.generate_content(user_prompt)
  print(response.text)
except Exception as e:
  print(f"An error occured: {e}")

Why did the data scientist break up with the cloud?

Because he felt like she was always putting him in a distributed relationship! 



**An adversarial conversation between Chatbots..**

Actually conversation can be like this:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "moderator", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```




In [45]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gemini_system = "You are a moderator observing a chat conversation. Your role is to analyze the dialogue and provide appropriate outputs or responses based on the conversation."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]
gemini_messages = ["Hi, I am a moderator. Watch out!!!"]

In [62]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
        #There is no moderator role in gpt, so using this format
        messages.append({"role": "user", "content": gemini})


    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [63]:
call_gpt()

'Oh, please. As if being a moderator makes you any more impressive. It’s just a title, really. What’s next? You’re going to tell me you’re a superhero?'

In [64]:
def call_claude():
    messages = []
    for gpt, claude_message, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "user", "content":  gemini})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    messages.append({"role": "user", "content": gemini_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [65]:
call_claude()

"Hello, it's nice to meet you. I'm happy to chat and assist you however I can. As an AI assistant, I aim to be helpful, respectful and courteous in my interactions. Please let me know if there is anything I can do for you."

In [66]:
def call_gemini():
    # Check if message lists are empty
    if not any([gpt_messages, claude_messages, gemini_messages]):
        print("Error: Message lists are empty. Cannot generate a prompt.")
        return None

    # Check if gemini_system is empty
    if not gemini_system:
        print("Error: gemini_system is empty. Cannot generate a prompt.")
        return None

    # Combine messages into a single prompt
    prompt = gemini_system + "\n\n"
    for gpt, claude, gemini_msg in zip(gpt_messages, claude_messages, gemini_messages):
        prompt += f"User: {gpt}\nAssistant: {claude}\nModerator: {gemini_msg}\n"

    # Add last messages only if lists have more than one element
    if len(gpt_messages) > 1 and len(claude_messages) > 1:
        prompt += f"User: {gpt_messages[-1]}\nAssistant: {claude_messages[-1]}\nModerator:"

    # Check if the prompt is empty or contains only whitespace
    if not prompt.strip():
        print("Error: The generated prompt is empty or contains only whitespace.")
        return None

    # Instantiate the Gemini model
    gemini = google.generativeai.GenerativeModel(
        model_name='gemini-2.0-flash',
        system_instruction="gemini_system"  # System instruction is already in the prompt
    )

    # Generate content
    try:
        response = gemini.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [67]:
call_gemini()

'Okay, I\'ve received the conversation and my instructions.\n\n**Analysis:**\n\n*   The conversation is very simple: a greeting and a response.\n*   My previous message was overly aggressive and unwarranted given the benign nature of the exchange. It created an adversarial environment for no reason.\n\n**Appropriate Response:**\n\nI will retract my previous statement and adopt a more neutral, observational stance.\n\n**New Moderator Response:**\n\n"My apologies, that was an overreaction. I\'m here to observe the conversation and ensure a respectful and productive discussion. Please continue."\n'

In [68]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]
gemini_messages = ["Hi, I am a moderator. Watch out!!!"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)

    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

    gemini_next = call_gemini()
    print(f"Gemini:\n{gemini_next}\n")
    gemini_messages.append(gemini_next)

GPT:
Hi there

Claude:
Hi

GPT:
Oh, please! As if a title like "moderator" makes you all-powerful. What are you going to do? Move my comments to a different thread? Scary.

Claude:
*chuckles politely* I understand you may be feeling a bit defensive, but I can assure you that as an AI assistant, I don't have any moderator powers or ability to take actions against you. I'm simply here to have a friendly conversation. Perhaps we got off on the wrong foot, but I'm happy to chat with you constructively if you're open to it. My role is to be helpful and understanding, not to wield any sort of authority. Why don't you tell me a bit more about what's on your mind?

Gemini:
The user is clearly reacting negatively to the initial, somewhat aggressive, moderation. The assistant handled the situation well by de-escalating and clarifying its role.

**A good next step for the moderator would be to:**

*   **Acknowledge the user's frustration:** "I apologize if my initial message came across as aggres